In [1]:
cd sec_parse

C:\Users\36036\OneDrive\Github\sec_edgar_parser\sec_parse


In [2]:
from sqlalchemy import distinct

In [6]:
from db import *
from config import *

In [3]:
db = EdgarDatabase()

In [4]:
db.make_session()

In [5]:
import pandas as pd

In [13]:
company_df = pd.read_sql_table(DB_COMPANY_TABLE, db._db_eng)
filing_info_df = pd.read_sql_table(DB_FILING_TABLE, db._db_eng, parse_dates=['period', 'filed'])
filing_data_df = pd.read_sql_table(DB_FILING_DATA_TABLE, db._db_eng, parse_dates=['value_period'])

In [ ]:
most_data_df = pd.merge(filing_data_df, filing_info_df, how='left')
all_data_df = pd.merge(most_data_df, company_df, how='left')

In [16]:
db.session.commit()

In [20]:
db.session.close()

In [7]:
db.session.query(distinct(FilingInfo.filing_accession)).filter(exists().where(FilingInfo.filing_accession == '0001193125-18-122816')).first()

('0000008947-18-000045')

In [11]:
r = db.session.query(distinct(FilingInfo.filing_accession)).filter(exists().where(FilingInfo.filing_accession == '001193125-18-122816')).first()

In [12]:
r

In [13]:
r is None

True

In [15]:
db.session.rollback()

In [33]:
r = db.session.query(FilingInfo, CompanyInfo).all()

In [30]:
r

(<db.FilingInfo at 0x1d54f6d67f0>, <db.CompanyInfo at 0x1d54f7eb390>)

In [34]:
len(r)

418134

In [22]:
import pandas as pd
import re
from typing import List, Union
from pathlib import Path

In [25]:
def _build_filing_dfs(file_path, re_search_terms) -> Union[None, List[pd.DataFrame]]:
    if not file_path:
        return None

    return_dfs = []

    if file_path.split(".")[-1] == 'xls' or 'xlsx':
        excel = pd.ExcelFile(file_path)

        for sheet_name in excel.sheet_names:
            if re.search(re_search_terms, sheet_name, flags=re.IGNORECASE):
                return_dfs.append(pd.read_excel(excel, sheet_name, header=None).dropna(how='all'))

    elif file_path.split(".")[-1] == 'csv':
        df_csv = pd.read_csv(file_path, header=None).dropna(how='all')

        header_vals_list = [str(item) for item in flatten(df_csv.iloc[:5, :].values.tolist())]
        header_vals_string = ' '.join([str(val) for val in header_vals_list])

        if not re.search(re_search_terms, header_vals_string):
            return_dfs.append(None)
        else:
            return return_dfs.append(df_csv)

    return return_dfs


def _clean_data_file(df, re_search_filing_type):
    if type(df) is None:
        return None

    header_vals_list = [str(item) for item in flatten(df.iloc[:5, :].values.tolist())]
    header_vals_string = ' '.join([str(val) for val in header_vals_list])

    # unit correction to 1 USD
    if re.search('thousands|Thousands', header_vals_string):
        unit_multiplier = 1000
    elif re.search('millions|Millions', header_vals_string):
        unit_multiplier = 1000000
    elif re.search('billions|Billions', header_vals_string):
        unit_multiplier = 1000000000
    else:
        unit_multiplier = 1

    # confirm sheet type by looking at header values for relevant string pattern
    if re.search(re_search_filing_type, header_vals_string, flags=re.IGNORECASE) is None:
        return None

    dropped_df = df.dropna()

    cleaned_df = dropped_df.applymap(lambda x: str(x).replace('\n', ' ').replace("'", "").replace(":", "")
                                     .replace('-', '').replace('*', '').replace('  ', ' ').strip().title())

    for col in cleaned_df.columns[1:]:
        cleaned_df.loc[1:, col] = cleaned_df.loc[1:, col].apply(scale_array_val, args=(unit_multiplier,))

    final_df = cleaned_df.dropna()

    return final_df.values

In [24]:
from numpy import nan
def scale_array_val(val, scale_val):
    try:
        return float(str(val).replace(',', ''))*scale_val
    except (ValueError, IndexError):
        if val == '':
            return nan
        else:
            return val

In [29]:
def flatten(deep_list):
    return [item for sublist in deep_list for item in sublist]

In [26]:
dfs = _build_filing_dfs("C:/Users/36036/sec_parse_data/xlsx_data/0001138978_0001493152-18-005063.xlsx", re_search_terms=r'\bstate.*?\bope|\bcond.*?\bconso')

In [27]:
len(dfs)

4

In [32]:
_clean_data_file(dfs[2], re_search_filing_type=r'condensed consolidated statements of operations')

array([['Revenues', 2145919.0, 1834620.0, 4399656.0, 3652759.0],
       ['Cost Of Revenues', 1307596.0, 1189908.0, 2715289.0, 2354021.0],
       ['Gross Profit', 838323.0, 644712.0, 1684367.0, 1298738.0],
       ['Selling Expenses', 30601.0, 8259.0, 68740.0, 18560.0],
       ['General And Administrative Expenses', 1934479.0, 589378.0,
        2914754.0, 1170242.0],
       ['Total Operating Expenses', 1965080.0, 597637.0, 2983494.0,
        1188802.0],
       ['Income (Loss) From Operations', 1126757.0, 47075.0, 1299127.0,
        109936.0],
       ['Interest Income', 160.0, 11010.0, 211.0, 21988.0],
       ['Interest Expense', 304132.0, 114617.0, 438285.0, 231705.0],
       ['Total Other Income (Expense)', 303972.0, 103607.0, 438074.0,
        209717.0],
       ['Loss Before Income Taxes', 1430729.0, 56532.0, 1737201.0,
        99781.0],
       ['Net Loss', 1376513.0, 56532.0, 1737201.0, 99781.0],
       ['Net Loss Attributed To Noncontrolling Interest', 1956.0, 2764.0,
        5356.0,

In [127]:
excel = pd.ExcelFile("C:/Users/36036/sec_parse_data/xlsx_data/0001138978_0001493152-18-005063.xlsx")
for sheet_name in excel.sheet_names:
    if re.search(r'\bstate.*?\bope', sheet_name, flags=re.I):
        print(sheet_name)

In [153]:
db.session.close()

In [38]:
dfs[2].head()

,0,1,2,3,4
0,Condensed Consolidated Statements of Operation...,3 Months Ended,NaN,6 Months Ended,NaN
1,NaN,"Feb. 28, 2018","Feb. 28, 2017","Feb. 28, 2018","Feb. 28, 2017"
2,Income Statement [Abstract],NaN,NaN,NaN,NaN
3,Revenues,2145919,1834620,4399656,3652759
4,Cost of revenues,1307596,1189908,2715289,2354021


In [56]:
scale_array_val(dropped_df.loc[1,1], 1)

2018.0

In [51]:
df = dfs[2]

header_vals_list = [str(item) for item in flatten(dfs[2].iloc[:5, :].values.tolist())]
header_vals_string = ' '.join([str(val) for val in header_vals_list])

# unit correction to 1 USD
if re.search('thousands|Thousands', header_vals_string):
    unit_multiplier = 1000
elif re.search('millions|Millions', header_vals_string):
    unit_multiplier = 1000000
elif re.search('billions|Billions', header_vals_string):
    unit_multiplier = 1000000000
else:
    unit_multiplier = 1

# confirm sheet type by looking at header values for relevant string pattern
if re.search(r'operations', header_vals_string, flags=re.I) is None:
    print('None')
    
if re.search(r'12 months', header_vals_string, flags=re.I) is None:
    print('None')
    
dropped_df = df.dropna(how='any', subset=df.columns[1:])

cleaned_df = dropped_df.applymap(lambda x: str(x).replace('\n', ' ').replace("'", "").replace(":", "")
                                 .replace('-', '').replace('*', '').replace('  ', ' ').strip().title())

for col in cleaned_df.columns[1:]:
    cleaned_df.loc[1:, col] = cleaned_df.loc[1:, col].apply(scale_array_val, args=(1,))

final_df = cleaned_df.dropna()

print(final_df.values)

None
[['Nan' 2018.0 2017.0 2018.0 2017.0]
 ['Revenues' 2145919.0 1834620.0 4399656.0 3652759.0]
 ['Cost Of Revenues' 1307596.0 1189908.0 2715289.0 2354021.0]
 ['Gross Profit' 838323.0 644712.0 1684367.0 1298738.0]
 ['Selling Expenses' 30601.0 8259.0 68740.0 18560.0]
 ['General And Administrative Expenses' 1934479.0 589378.0 2914754.0
  1170242.0]
 ['Total Operating Expenses' 1965080.0 597637.0 2983494.0 1188802.0]
 ['Income (Loss) From Operations' 1126757.0 47075.0 1299127.0 109936.0]
 ['Interest Income' 160.0 11010.0 211.0 21988.0]
 ['Interest Expense' 304132.0 114617.0 438285.0 231705.0]
 ['Total Other Income (Expense)' 303972.0 103607.0 438074.0 209717.0]
 ['Loss Before Income Taxes' 1430729.0 56532.0 1737201.0 99781.0]
 ['Net Loss' 1376513.0 56532.0 1737201.0 99781.0]
 ['Net Loss Attributed To Noncontrolling Interest' 1956.0 2764.0 5356.0
  4491.0]
 ['Net Loss Attributed To Novo Integrated Sciences, Inc.' 1374557.0
  53768.0 1731845.0 95290.0]
 ['Net Loss' 1376513.0 56532.0 1737201

In [59]:
df.drop_duplicates(subset=[0], keep=False, inplace=True)